### Data processing for xBD Dataset ###
This notebook handles all the required cropping and data processing for the xBD dataset. It takes in raw satellite images and outputs crops of buildings labeled as either damaged or undamaged.

In [1]:
import rasterio
from rasterio.windows import from_bounds
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import re

In [4]:
# find all disasters and list names in xBD tier 1 dataset
filepath = 'xBD_Dataset/geotiffs/tier1/images'
contents = os.listdir(filepath)
disaster_names = set()
for i in contents:
    disaster_names.add(re.findall(r"([\w,-]+)_\d", i)[0])
print(disaster_names)

{'guatemala-volcano', 'midwest-flooding', 'hurricane-harvey', 'santa-rosa-wildfire', 'hurricane-michael', 'hurricane-florence', 'hurricane-matthew', 'palu-tsunami', 'socal-fire'}


In [5]:
# find all disasters and list names in xBD tier 3 dataset
filepath = 'xBD_Dataset/geotiffs/tier3/images'
contents = os.listdir(filepath)
disaster_names = set()
for i in contents:
    disaster_names.add(re.findall(r"([\w,-]+)_\d", i)[0])
print(disaster_names)

{'moore-tornado', 'pinery-bushfire', 'joplin-tornado', 'tuscaloosa-tornado', 'lower-puna-volcano', 'woolsey-fire', 'sunda-tsunami', 'portugal-wildfire', 'nepal-flooding'}


In [6]:
# to better match our use case, apply transfer learning only on the following datasets:
usable_disasters = ['santa-rosa-wildfire', 'socal-fire', 'joplin-tornado','woolsey-fire','tuscaloosa-tornado', 'moore-tornado', 'portugal-wildfire']

In [10]:
# the original directory already contains training, hold, and test folders
# however, we want to create our own train/validation/test splits
# therefore, we grab all images from the three folders (tier1, hold, test) and extract buildings

# create counter mechanism for names of cropped images
counter = dict()
for name in usable_disasters:
    counter.update({name:0})

for folder in ['xBD_Dataset/geotiffs/tier1','xBD_Dataset/geotiffs/tier3','xBD_Dataset/geotiffs/hold','xBD_Dataset/geotiffs/test']:
    contents = os.listdir(folder + '/labels')
    for label_path in contents:
        current_disaster = re.findall(r"([\w,-]+)_\d", label_path)[0]
        if (current_disaster in usable_disasters) and ('post' in label_path):
            image_path = label_path[:-5] + '.tif'
            
            label = pd.read_json(folder + '/labels/' + label_path)['features']['xy']
            df = pd.json_normalize(label)
            for j in range(df.shape[0]):
                coords = df.loc[j,'wkt']
                uid = df.loc[j,'properties.uid']
                damage_type = df.loc[j,'properties.subtype']
                
                # use regex to extract two groups corresponding to the pixel x and y values of the building
                result = re.findall(r"(\d+\.\d+)\s(\d+\.\d+)", coords)

                xvals = []
                yvals = []
                for i in result:
                    xvals.append(float(i[0]))
                    yvals.append(float(i[1]))

                ymax = max(yvals) + 10
                ymin = min(yvals) - 10
                xmax = max(xvals) + 10
                xmin = min(xvals) - 10
                
                # discard this building if too small for human to make reliable prediction
                if ((ymax - ymin) < 40 or (xmax - xmin) < 40):
                    continue

                with rasterio.open(folder + '/images/' + image_path) as src:
                    # aff = src.transform
                    # Make sure not extracting pixels that are out of bounds
                    rxmin, rymin, rxmax, rymax = (0, 0, src.shape[1], src.shape[0])
                    ymax = min(ymax,rymax)
                    ymin = max(ymin,rymin)
                    xmax = min(xmax,rxmax)
                    xmin = max(xmin,rxmin)

                    window = ((int(ymin), int(ymax)), (int(xmin), int(xmax)))
                    # Read croped array
                    arr1 = src.read(1, window=window)
                    arr2 = src.read(2, window=window)
                    arr3 = src.read(3, window=window)

                # output image of cropped building
                rgb = np.dstack((arr1,arr2,arr3)).astype(np.uint8)
                new_image = Image.fromarray(rgb)
                
                if damage_type in ['no-damage','minor-damage']:
                    damage_status = 'undamaged'
                else:
                    damage_status = 'damaged'
                
                counter[current_disaster] += 1
                save_path = './xBD_Dataset/' + damage_status + '/'
                save_name = current_disaster + '-' + str(counter[current_disaster]) + '-' + damage_status + '.png'
                new_image.save(save_path + save_name)
                

In [11]:
total = 0
for key, val in counter.items():
    print('Processed ' + str(val) + ' buildings for disaster ' + key)
    total += val
print('Processed ' + str(total) + ' buildings in total')

Processed 17676 buildings for disaster santa-rosa-wildfire
Processed 13491 buildings for disaster socal-fire
Processed 9640 buildings for disaster joplin-tornado
Processed 3756 buildings for disaster woolsey-fire
Processed 10681 buildings for disaster tuscaloosa-tornado
Processed 16881 buildings for disaster moore-tornado
Processed 14657 buildings for disaster portugal-wildfire
Processed 86782 buildings in total


In [4]:
import glob
widths = []
heights = []
for name in glob.glob('./xBD_Dataset/building_crops/*/*'):
    with Image.open(name) as img:
        widths.append(img.size[0])
        heights.append(img.size[1])

In [5]:
print(np.median(widths))
print(np.median(heights))

59.0
58.0


In [6]:
print(np.mean(widths))
print(np.mean(heights))

62.68584499089673
61.75250628010417


In [10]:
widths.shape()

86782

In [2]:
# to better match our use case, apply transfer learning only on the following datasets:
testing_disasters = ['pinery-bushfire']

In [4]:
# create counter mechanism for names of cropped images
counter = dict()
for name in testing_disasters:
    counter.update({name:0})

for folder in ['xBD_Dataset/geotiffs/tier1','xBD_Dataset/geotiffs/tier3','xBD_Dataset/geotiffs/hold','xBD_Dataset/geotiffs/test']:
    contents = os.listdir(folder + '/labels')
    for label_path in contents:
        current_disaster = re.findall(r"([\w,-]+)_\d", label_path)[0]
        if (current_disaster in testing_disasters) and ('post' in label_path):
            image_path = label_path[:-5] + '.tif'
            
            label = pd.read_json(folder + '/labels/' + label_path)['features']['xy']
            df = pd.json_normalize(label)
            for j in range(df.shape[0]):
                coords = df.loc[j,'wkt']
                uid = df.loc[j,'properties.uid']
                damage_type = df.loc[j,'properties.subtype']
                
                # use regex to extract two groups corresponding to the pixel x and y values of the building
                result = re.findall(r"(\d+\.\d+)\s(\d+\.\d+)", coords)

                xvals = []
                yvals = []
                for i in result:
                    xvals.append(float(i[0]))
                    yvals.append(float(i[1]))

                ymax = max(yvals) + 10
                ymin = min(yvals) - 10
                xmax = max(xvals) + 10
                xmin = min(xvals) - 10
                
                # discard this building if too small for human to make reliable prediction
                if ((ymax - ymin) < 40 or (xmax - xmin) < 40):
                    continue

                with rasterio.open(folder + '/images/' + image_path) as src:
                    # aff = src.transform
                    # Make sure not extracting pixels that are out of bounds
                    rxmin, rymin, rxmax, rymax = (0, 0, src.shape[1], src.shape[0])
                    ymax = min(ymax,rymax)
                    ymin = max(ymin,rymin)
                    xmax = min(xmax,rxmax)
                    xmin = max(xmin,rxmin)

                    window = ((int(ymin), int(ymax)), (int(xmin), int(xmax)))
                    # Read croped array
                    arr1 = src.read(1, window=window)
                    arr2 = src.read(2, window=window)
                    arr3 = src.read(3, window=window)

                # output image of cropped building
                rgb = np.dstack((arr1,arr2,arr3)).astype(np.uint8)
                new_image = Image.fromarray(rgb)
                
                if damage_type in ['no-damage','minor-damage']:
                    damage_status = 'undamaged'
                else:
                    damage_status = 'damaged'
                
                counter[current_disaster] += 1
                save_path = './xBD_Dataset/' + damage_status + '/'
                save_name = current_disaster + '-' + str(counter[current_disaster]) + '-' + damage_status + '.png'
                new_image.save(save_path + save_name)
total = 0
for key, val in counter.items():
    print('Processed ' + str(val) + ' buildings for disaster ' + key)
    total += val
print('Processed ' + str(total) + ' buildings in total')

Processed 2353 buildings for disaster pinery-bushfire
Processed 2353 buildings in total
